In [1]:
import json
import math

In [2]:
results_path = "results.json"

results_optimized_par_path = "results_optimized_par.json"

In [3]:
with open(results_path, "r") as f:
    results = json.load(f)
    
with open(results_optimized_par_path, "r") as f:
    results_optimized_par = json.load(f)

In [4]:
tol = 1e-4

# time以外の結果がほぼ一致しているかを確認
for key in results.keys():
    for method in results[key].keys():
        for metric in results[key][method].keys():
            if metric == "time":
                continue

            orig = results[key][method][metric]
            test = results_optimized_par[key][method][metric]

            def compare(o, t, path):
                # 数値なら isclose
                if isinstance(o, (int, float)):
                    assert math.isclose(o, t, rel_tol=tol, abs_tol=tol), \
                        f"{path} mismatch: {o} vs {t}"
                # リスト／タプルなら要素ごとに再帰チェック
                elif isinstance(o, (list, tuple)):
                    assert len(o) == len(t), \
                        f"{path} length mismatch: {len(o)} vs {len(t)}"
                    for idx, (oi, ti) in enumerate(zip(o, t)):
                        compare(oi, ti, f"{path}[{idx}]")
                # それ以外は厳密一致
                else:
                    assert o == t, f"{path} mismatch: {o} vs {t}"

            compare(orig, test, f"{key}.{method}.{metric}")

print("time以外はすべて許容範囲内で一致しました。")

AssertionError: M5_delta0.6.ebpa3.sales_ratio[1] mismatch: 0.9638150661089137 vs 0.9493790423822295

In [8]:
import math
import numpy as np # NumPy の数値型をチェックするためにインポート

# results と results_optimized_par は事前にロードされていると仮定します。
# 例:
# with open('results.json', 'r') as f:
#     results = json.load(f)
# with open('results_optimized_par.json', 'r') as f:
#     results_optimized_par = json.load(f)

tol = 1e-1
mismatched_records = [] # 不一致情報を記録するリスト

def compare(o, t, path, records):
    """
    二つの値を再帰的に比較し、許容誤差を超える不一致があれば記録する関数。

    Args:
        o: オリジナルの値
        t: テスト対象の値
        path: 現在比較しているデータのパス (デバッグ用)
        records: 不一致記録用のリスト
    """
    # --- 型チェックと数値比較 ---
    # オリジナルが数値の場合
    if isinstance(o, (int, float)) or isinstance(o, np.number):
        # テスト対象も数値か確認
        if isinstance(t, (int, float)) or isinstance(t, np.number):
            # math.isclose で比較し、Falseなら記録
            if not math.isclose(o, t, rel_tol=tol, abs_tol=tol):
                records.append({
                    "path": path,
                    "original": o,
                    "test": t,
                    "diff": abs(o - t) # 差の絶対値も記録
                })
        else:
            # 型が不一致の場合も記録
            records.append({
                "path": path,
                "original_value": o,
                "test_value": t,
                "error": f"Type mismatch (expected numeric, got {type(t)})"
            })
        return # 数値比較が終わったらこのパスは終了

    # --- リスト/タプルの比較 ---
    elif isinstance(o, (list, tuple)):
        # テスト対象もリスト/タプルか確認
        if isinstance(t, (list, tuple)):
            # 長さが違う場合は記録
            if len(o) != len(t):
                records.append({
                    "path": path,
                    "original_len": len(o),
                    "test_len": len(t),
                    "error": "Length mismatch"
                })
            else:
                # 長さが同じなら要素ごとに再帰比較
                for idx, (oi, ti) in enumerate(zip(o, t)):
                    compare(oi, ti, f"{path}[{idx}]", records)
        else:
            # 型が不一致の場合も記録
            records.append({
                "path": path,
                "original_type": type(o),
                "test_type": type(t),
                "error": f"Type mismatch (expected list/tuple, got {type(t)})"
            })
        return # リスト/タプル比較が終わったらこのパスは終了

    # --- その他の型の厳密比較 ---
    # (ここでは主に文字列などを想定)
    else:
        if o != t:
            records.append({
                "path": path,
                "original": o,
                "test": t,
                "error": "Exact value mismatch"
            })

# --- メインの比較ループ ---
print("比較を開始します...")

for key in results.keys():
    # results_optimized_par にも同じ key が存在するかチェック
    if key not in results_optimized_par:
        print(f"警告: キー '{key}' が results_optimized_par に見つかりません。スキップします。")
        continue

    # 'r2_list' のようなトップレベルのリストを処理
    if 'r2_list' in results[key]:
         if 'r2_list' in results_optimized_par[key]:
             orig_r2 = results[key]['r2_list']
             test_r2 = results_optimized_par[key]['r2_list']
             compare(orig_r2, test_r2, f"{key}.r2_list", mismatched_records)
         else:
             print(f"警告: メトリック 'r2_list' が results_optimized_par['{key}'] に見つかりません。スキップします。")


    for method in results[key].keys():
        # method が 'r2_list' の場合は上で処理済みなのでスキップ
        if method == 'r2_list':
            continue

        # results_optimized_par[key] にも同じ method が存在するかチェック
        if method not in results_optimized_par[key]:
            print(f"警告: メソッド '{method}' が results_optimized_par['{key}'] に見つかりません。スキップします。")
            continue

        for metric in results[key][method].keys():
            # results_optimized_par[key][method] にも同じ metric が存在するかチェック
            if metric not in results_optimized_par[key][method]:
                print(f"警告: メトリック '{metric}' が results_optimized_par['{key}']['{method}'] に見つかりません。スキップします。")
                continue

            if metric == "time":
                continue

            orig = results[key][method][metric]
            test = results_optimized_par[key][method][metric]

            # compare 関数を呼び出し、記録用リストを渡す
            compare(orig, test, f"{key}.{method}.{metric}", mismatched_records)

# --- 結果の表示 ---
mismatch_count = len(mismatched_records)

print("\n--- 比較結果 ---")
if mismatch_count == 0:
    print("time以外のすべての項目が許容範囲内で一致しました。")
else:
    print(f"許容誤差 ({tol}) を超える不一致、または型の不一致が {mismatch_count} 件見つかりました。")

    # オプション: 不一致の詳細を表示 (件数が多い場合は最初の10件などに制限)
    print("\n不一致の詳細 (最初の10件):")
    for i, record in enumerate(mismatched_records[:10]):
        print(f"  {i+1}. Path: {record['path']}")
        if 'error' in record:
             print(f"     Error: {record['error']}")
             # エラーによっては値も表示
             if 'original_value' in record: print(f"     Original: {record['original_value']}")
             if 'test_value' in record: print(f"     Test: {record['test_value']}")
             if 'original_len' in record: print(f"     Original Length: {record['original_len']}")
             if 'test_len' in record: print(f"     Test Length: {record['test_len']}")
             if 'original_type' in record: print(f"     Original Type: {record['original_type']}")
             if 'test_type' in record: print(f"     Test Type: {record['test_type']}")

        else: # isclose で False になった場合
             print(f"     Original: {record['original']}")
             print(f"     Test: {record['test']}")
             print(f"     Difference: {record['diff']:.6g}") # 差も表示

print("----------------")

比較を開始します...

--- 比較結果 ---
許容誤差 (0.1) を超える不一致、または型の不一致が 3827 件見つかりました。

不一致の詳細 (最初の10件):
  1. Path: M5_delta0.6.ebpa3.range_diff[1]
     Original: 0.23898810629746037
     Test: 0.47569203727968756
     Difference: 0.236704
  2. Path: M5_delta0.6.ebpa3.range_diff[25]
     Original: 0.7566495585009853
     Test: 0.9515858197826621
     Difference: 0.194936
  3. Path: M5_delta0.6.ebpa3.range_diff[29]
     Original: 0.7312239469925013
     Test: 0.42669214188371785
     Difference: 0.304532
  4. Path: M5_delta0.6.ebpa3.range_diff[37]
     Original: 1.2531254919057517
     Test: 0.9108064600318881
     Difference: 0.342319
  5. Path: M5_delta0.6.ebpa3.range_diff[52]
     Original: 0.5125398585884204
     Test: 1.4994788988915837
     Difference: 0.986939
  6. Path: M5_delta0.6.ebpa3.range_diff[60]
     Original: 0.41154841955767707
     Test: 0.30300659871737845
     Difference: 0.108542
  7. Path: M5_delta0.6.ebpa3.range_diff[67]
     Original: 1.331581390967301
     Test: 1.49962416814316